In [3]:
# build a brevitas binary neural network for MNIST
import torch
import torch.nn as nn
import brevitas.nn as qnn
from brevitas.quant import Int8WeightPerTensorFloat, SignedBinaryWeightPerTensorConst
from brevitas.quant import Int8ActPerTensorFloat, SignedBinaryActPerTensorConst

class SignedBinaryWeightPerTensorConstOne(SignedBinaryWeightPerTensorConst):
    scaling_const = 1

class BinaryNet(nn.Module):
    def __init__(self):
        super(BinaryNet, self).__init__()
        
        # 輸入層到第一個隱藏層
        self.fc1 = qnn.QuantLinear(
            5,  # MNIST 輸入維度 (28x28)
            5,
            bias=True,
            input_quant=SignedBinaryActPerTensorConst,
            weight_quant=SignedBinaryWeightPerTensorConstOne
        )
        
        # 第一個隱藏層到第二個隱藏層
        self.fc2 = qnn.QuantLinear(
            5,
            5,
            bias=True, 
            input_quant=SignedBinaryActPerTensorConst,
            weight_quant=SignedBinaryWeightPerTensorConstOne
        )

        self.quant_relu = qnn.QuantReLU()
        
        
    def forward(self, x):
        
        # 前向傳播
        x = self.quant_relu(self.fc1(x))
        x = self.fc2(x)
        
        return x



In [4]:
from hls4ml.converters import convert_from_pytorch_model
from hls4ml.utils.config import config_from_pytorch_model
# 初始化 BinaryNet 模型
model = BinaryNet()

# 設定輸入形狀
input_shape = (None, 5)  # MNIST 數據集的形狀

# 從模型生成配置
config = config_from_pytorch_model(model, backend='Vitis')

# 設定輸出目錄
output_dir = 'hls4ml_bnn_prj'

# 轉換為 HLS 模型
hls_model_init = convert_from_pytorch_model(
    model,
    input_shape,
    hls_config=config,
    output_dir=output_dir,
    backend='Vitis',
    io_type='io_parallel'
)

# 編譯 HLS 模型
hls_model_init.compile()



Interpreting Model ...
dict_keys(['Conv1d', 'QuantConv1d', 'Conv2d', 'QuantConv2d', 'cat', 'concat', 'concatenate', 'add', 'mul', 'multiply', 'sub', 'subtract', 'fmin', 'minimum', 'fmax', 'maximum', 'View', 'squeeze', 'unsqueeze', 'Flatten', 'Upsample', 'UpsamplingNearest2d', 'UpsamplingBilinear2d', 'Linear', 'QuantLinear', 'Softmax', 'ReLU', 'LeakyReLU', 'Threshold', 'ELU', 'PReLU', 'Sigmoid', 'Tanh', 'QuantReLU', 'QuantSigmoid', 'QuantTanh', 'BatchNorm2d', 'BatchNorm1d', 'Batch_norm', 'MaxPool1d', 'MaxPool2d', 'AvgPool1d', 'AvgPool2d', 'QuantMaxPool1d', 'QuantMaxPool2d'])
Topology:
Layer name: fc1, layer type: Dense, input shape: [[None, 10]]
Layer name: quant_relu, layer type: Activation, input shape: [[None, 10]]
Layer name: fc2, layer type: Dense, input shape: [[None, 10]]
Creating HLS model
Writing HLS project
Done


In [5]:
# 生成隨機輸入數據
import torch
import numpy as np

# 生成一個隨機輸入張量
x = torch.randn(1, 10)  # 批次大小為1,784個特徵
# 使用PyTorch模型進行預測
pytorch_prediction = model(x).detach().numpy()

# 使用HLS模型進行預測
hls_prediction = hls_model_init.predict(x.numpy())

# 重塑HLS預測結果以匹配PyTorch預測的形狀
hls_prediction = np.reshape(hls_prediction, pytorch_prediction.shape)

# 比較兩個模型的預測結果
print("PyTorch模型預測結果:")
print(pytorch_prediction)
print("\nHLS模型預測結果:")
print(hls_prediction)

# 計算預測結果的相對誤差
np.testing.assert_allclose(hls_prediction, pytorch_prediction, rtol=1e-2, atol=0.01)
print("\n兩個模型的預測結果在允許的誤差範圍內相符")


PyTorch模型預測結果:
[[ 3.9261074  2.183936  -1.9792562  1.927779   2.1620696 -4.2387676
  -3.965987   3.8596807  2.0498013  1.8720356]]

HLS模型預測結果:
[[-5.6972656 -5.439453  -5.602539  -5.6953125 -5.461914  -5.8623047
  -5.5898438 -5.763672  -5.5742188 -5.751953 ]]


/home/robin/miniconda3/envs/torch/lib/python3.8/site-packages/torch/_tensor.py:1413: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1925.)
  return super().rename(names)


AssertionError: 
Not equal to tolerance rtol=0.01, atol=0.01

Mismatched elements: 10 / 10 (100%)
Max absolute difference: 9.623373
Max relative difference: 4.072566
 x: array([[-5.697266, -5.439453, -5.602539, -5.695312, -5.461914, -5.862305,
        -5.589844, -5.763672, -5.574219, -5.751953]], dtype=float32)
 y: array([[ 3.926107,  2.183936, -1.979256,  1.927779,  2.16207 , -4.238768,
        -3.965987,  3.859681,  2.049801,  1.872036]], dtype=float32)

In [11]:
# 打印PyTorch模型第一層量化後的權重
quant_weight = model.fc1.quant_weight()
print("第一層量化後的權重:")
print(quant_weight)

import numpy as np
hls_weight = np.array([-1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1])
print("HLS模型第一層量化後的權重:")
print(hls_weight.reshape(10, 10))



第一層量化後的權重:
QuantTensor(value=tensor([[-1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.],
        [ 1., -1., -1.,  1., -1.,  1.,  1., -1., -1., -1.],
        [ 1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.],
        [ 1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.],
        [-1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.],
        [-1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.],
        [-1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.],
        [ 1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1.]],
       grad_fn=<MulBackward0>), scale=tensor(1.), zero_point=tensor(0.), bit_width=tensor(1.), signed_t=tensor(True), training_t=tensor(True))
HLS模型第一層量化後的權重:
[[-1  1 -1  1  1  1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1  1 -1  1  1  1]
 [-1  1 -1 -1 -1 -1  1 -1 -1  1]
 [-1  1  1  1  1  1 -1  1  1 -1]
 [ 1 -1  1 -1 -1 -1 -1  1  1 -1]
 [ 1 -1 -1  1 -1  1 -1 -1 -1  1]
 [ 1 -1  1  1